In [2]:
import numpy as np
import pandas as pd

In [7]:
finance_news = pd.read_csv('news.csv', encoding = 'ISO-8859-1')
finance_news.head()

,sentiment,headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [8]:
X = finance_news['headline'].to_list()
y = finance_news['sentiment'].to_list()

for i,x in enumerate(X):
    print(i+1,")",x)

1 ) According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
2 ) Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .
3 ) The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .
4 ) With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
5 ) According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
6 ) FINANCING OF ASPO

728 ) 9 September 2010 - Finnish stationery and gift retailer Tiimari HEL : TII1V said today its net sales rose by 2 % year-on-year to EUR5 .7 m in August 2010 , driven by growth in Finland , while demand in the Baltics remained weak .
729 ) By 14:29 CET on Monday , shares in Bavarian Nordic had climbed 1.21 % to DKK250 on the stock exchange in Copenhagen after having lost 7.41 % in the past month .
730 ) For example , net sales increased by 5.9 % from the first quarter , and EBITDA increased from a negative EUR 0.2 mn in the first quarter of 2009 .
731 ) Mobile phone sales rose 25 % to 5.87 billion euros , while enterprise solution sales dropped 39 % to 186 million euros .
732 ) RISING costs have forced packaging producer Huhtamaki to axe 90 jobs at its Hampshire manufacturing plant .
733 ) `` However , the rapidly increasing costs and the strengthening of the euro were challenging for the whole industry , and they impacted on our results . ''
734 ) At this growth rate , paying off th

1362 ) The company designs and manufactures high-quality clothing , interior decoration textiles , bags , and other accessories .
1363 ) The company has delivered the technical infrastructure , used by NAV for their data warehouse and business intelligence initiatives .
1364 ) The company is now withdrawing the second part , EUR 7.2 mn , of the investment commitment .
1365 ) The company is presently examining whether the project would be financially feasible .
1366 ) The company is studying the feasibility of focusing most of its processed meat production in the Vantaa facilities and the processing of fresh meat in the Forssa facilities .
1367 ) The company 's strength is its Apetit brand .
1368 ) The company said shareholders will be able to vote on the agreement at an EGM scheduled for later this month .
1369 ) The company specialises in temporary electrification and heating at construction sites .
1370 ) The competition was received with great enthusiasm by the employees , he goes o

2051 ) The item included restructuring costs of EUR1 .6 m , while a year earlier they were EUR13 .1 m. Diluted EPS stood at EUR0 .3 versus a loss per share of EUR 0.1 .
2052 ) The new office , located in Shenzhen , will strengthen Vaisala 's already 10-year old presence in China .
2053 ) The price of raw material aluminium went up at the end of 2005 , but the company considers its outlook for 2006 favourable .
2054 ) The refining margin for the year was $ 13.39 - bbl , compared to $ 10.46 - bbl in the prior year .
2055 ) The report also said Nokia 's operations comprising the sites at Bochum , Ulm , Dusseldorf , Munich and Frankfurt posted operating profit after interest of 246 mln eur .
2056 ) The subdivision made sales revenues last year of EUR 480.7 million EUR 414.9 million in 2008 , and operating profits of EUR 44.5 million EUR 7.4 million .
2057 ) The value of orders on hand totaled EUR 237.1 mn .
2058 ) The value of the confirmed aircraft orders totals EUR 2bn .
2059 ) Vaahto , 

2775 ) After buying Eukor Car Carriers 50 percent stake , Grimaldi Group is now the sole owner of the Swedish roll-on , roll-off port of Wallhamn .
2776 ) After completion of the acquisition , Poyry 's oil and gas expertise will cover multi-discipline engineering and consulting services across the complete oil and gas asset lifecycle , including deepwater engineering services .
2777 ) After the restructuring , UPM 's average paper machine capacity in Europe will be 320,000 tons ( 350,000 short tons ) in coated magazine paper and 420,000 tons ( 460,000 short tons ) in coated fine paper .
2778 ) After the transaction , M-real will own 30 % in Metsa-Botnia and UPM -- 17 % .
2779 ) After the transaction , Tikkurila has no powder coatings related operations .
2780 ) Ahlstrom Corporation STOCK EXCHANGE ANNOUNCEMENT 23.4.2007 Ahlstrom Corporation will publish its first quarter financial results 2007 on Friday , April 27 , 2007 approximately at 8.30 a.m. Finnish time .
2781 ) AHMS will also of

3462 ) YIT acquired investment rights to a 10,000 square metre residential project in Yaroslavl and to a 16,400 square metre project in Moscow .
3463 ) YIT CORPORATION SEPT. 24 , 2007 at 13:30 CORPORATE RELEASE STOCK EXCHANGE RELEASE YIT 'S CAPITAL MARKETS DAY IN LONDON , SEPT. 26 , 2007 On Wednesday , September 26 , 2007 , YIT will hold a Capital Markets Day for investors and analysts in London .
3464 ) YIT Kausta will build commercial and residential highrises in Virsuliskes area .
3465 ) You 're not alone .
3466 ) ---------------------------------------------------------------------- -------------- Munich , 14 January 2008 : BAVARIA Industriekapital AG closed the acquisition of Elcoteq Communications Technology GmbH in Offenburg , Germany , with the approval of the
3467 ) - Cash flow from operating activities before investments was EUR 0.8 -1.2 million .
3468 ) - Cash flow from operating activities before investments was EUR 7.6 million EUR 2.5 million .
3469 ) -- Christine Idzelis 

4121 ) Mr Clausen , however , refused to comment the option that Nordea would consider buying into Citadele Bank .
4122 ) Nokia and Capcom announced that Resident Evil Degeneration will be released on N-Gage later this year .
4123 ) Nokia will certainly disagree with Qualcomm 's views on the patent situation .
4124 ) Olli-Pekka Kallasvuo was elected as vice chairman of the Board .
4125 ) Olli-Pekka Laine has been appointed as the Chairman and Erkki Pehu-Lehtonen as the Vice Chairman of the Board .
4126 ) Outokumpu 's steel mill in Tornio , in Finland , is the suspected source .
4127 ) Possible personnel reductions concern approximately 104 people .
4128 ) Risk exposure by Non-life Insurance  Moving 12-month  Expenses by function in Non-life Insurance excluding expenses for investment management and expenses for other services rendered Non-life Insurance investment portfolio by allocation
4129 ) Secure your files online Like filling out a tax return , making a backup is boring .
4130 ) 

4811 ) Finnish meat company Atria can no longer promise a sufficient amount of domestic beef to its customers .
4812 ) Thanks to the internet , consumers compare products more than previously and Finnish companies are not competitive .
4813 ) Because expenditures must be justified to pass budget approval hurdles , we believe our RoP model can help make it easier for IT and IT security practitioners to make the business case for acquiring enabling security technologies and related control activities .
4814 ) `` Capital expenditure on energy efficiency has unfortunately fallen along with the decline in the economy .
4815 ) Due to the rapid decrease in net sales , personnel reductions have been carried out on a wider scale than initially expected .
4816 ) Operating profit excluding non-recurring items decreased to EUR 6.2 mn from EUR 16.8 mn in 2007 , representing 2.3 % of net sales .
4817 ) Finnish waste management and cleaning group Lassila & Tikanoja Oyj ( L&T ) net profit went down to

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [ ]:
sent_val = list()
for x in X:
    inputs = tokenizer(x, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
   
    val = labels[np.argmax(outputs.detach().numpy())]
    print(x, '----', val)
    print('#######################################################')    
    sent_val.append(val)

In [ ]:
len(sent_val)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y, sent_val))